# Étude de marché 🐔

Objectifs :
- Exporter les produits dans de nouveaux produits plutôt que de produire sur place
- Étudier les régimes alimentaires de tous les pays, notamment en termes de protéines d'origine animale et en termes de calories
- Cibler plus particulièrement certains pays, dans le but d'approfondir ensuite l'étude de marché (produire des "groupes" de pays, plus ou moins gros, dont on connaît les caractéristiques)
- Identifier les pays propices à une insertion dans le marché du poulet

In [ ]:
pip install pca

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import plotly.express as px

from sklearn.preprocessing import normalize # Normalisation
from sklearn.preprocessing import StandardScaler
from pca import pca # ACP
import scipy.cluster.hierarchy as shc # Dendrogramme
from sklearn.cluster import AgglomerativeClustering # Récupération des clusters
import scipy.stats as stats # Tests statistiques
from scipy.special import boxcox1p
from statsmodels.graphics.gofplots import qqplot
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# Couleurs
black = '#000'
red = '#d50000'
teal = '#00bfa5'
indigo = '#304ffe'
amber = '#ffab00'
purple = '#aa00ff'
palette = [red, teal, indigo, amber, purple, black]

# Exploration

## 4 jeux de données

- Sources (FAO) : 
    - [New Food Balances](http://www.fao.org/faostat/en/#data/FBS)
    - [Annual population](http://www.fao.org/faostat/en/#data/OA)
- Variables :
    - L'ensemble des pays disponibles
    - La différence de population entre une année antérieure et l'année courante
    - Disponibilité alimentaire en protéines par habitant
    - Disponibilité alimentaire en calories par habitant
    - Proportion de protéines d'origine animale par rapport à la quantité totale de protéines dans la disponibilité alimentaire 

In [ ]:
# URL sources
url = 'https://raw.githubusercontent.com/gllmfrnr/oc/master/p5/sources/fao_'

# Colonnes inutiles
drop_columns = ['Domain Code', 'Area Code', 'Element Code', 'Item Code', 'Flag', 'Flag Description', 'Year Code']

# Dataframes
food = pd.read_csv(url + 'new-food-balances.csv').drop(drop_columns, axis=1) # Nouveaux Bilans Alimentaires
livestock = pd.read_csv(url + 'livestock-primary.csv').drop(drop_columns, axis=1) # Élevage primaire
population = pd.read_csv(url + 'annual-population.csv').drop(drop_columns + ['Note'], axis=1) # Séries temporelles annuelles
security = pd.read_csv(url + 'security-indicators.csv').drop(drop_columns + ['Note'], axis=1) # Données de la sécurité alimentaire

In [ ]:
food.sample(5)

In [ ]:
livestock.sample(5)

In [ ]:
population.sample(5)

In [ ]:
security.sample(5)

## Jointures et pivot

In [ ]:
# Concaténation des 4 dataframes
data = pd.concat([food, livestock, population, security])
data.sample(5)

In [ ]:
# Table pivot
data = data.pivot_table(index='Area', values='Value', columns=['Element', 'Item', 'Year']).reset_index()
data

In [ ]:
# Abbréviations du multiIndex
export, imports, production, chicken, kcal, prot = 'Export Quantity', 'Import Quantity', 'Production', 'Meat, chicken', 'Food supply (kcal/capita/day)', 'Protein supply quantity (g/capita/day)'
pop = 'Total Population - Both sexes'
pop2 = 'Population - Est. & Proj.'
animal = 'Animal Products'
total = 'Grand Total'
secu = 'Value'
pib = 'Gross domestic product per capita, PPP, dissemination (constant 2011 international $)'
stable = 'Political stability and absence of violence/terrorism (index)'

# Années analysées
annee_a = 2008
annee_b = 2018

# Nouvelles variables
df = pd.DataFrame()
df['Pays'] = data['Area']
df['Dispo_calories'] = data[kcal][total][annee_b]
df['Dispo_protéines'] = data[prot][total][annee_b]
df['Ratio protéines animales'] = data[prot][animal][annee_b] / data[prot][total][annee_b]
df['Millions habitants'] = data[pop][pop2][annee_b] / 1000
df['Croissance population'] = data[pop][pop2][annee_b] / data[pop][pop2][annee_a]
df['PIB'] = data[secu][pib][annee_b]
df['Croissance PIB'] = data[secu][pib][annee_b] / data[secu][pib][annee_a]
df['Poulet imports / exports'] = data[imports][chicken][annee_b] / data[export][chicken][annee_b]

# Suppression des valeurs manquantes et infinies : restent 106 pays
df = df.replace([np.inf, -np.inf], np.nan).dropna().reset_index()
df = df.drop(['index'], axis=1)
df

# --> 106 pays au total (sans valeurs manquantes)

# ACP et hiérarchisation

## Corrélations
Fortes corrélations entre :
- la dispo. en protéines et la dispo. en calories
- la dispo. en protéines et le ratio de protéines animales
- la dispo. en protéines et le PIB

Nous sommes intéressés par les pays les plus susceptibles de consommer du poulet, donc ceux ayant un fort ratio de protéines animales. Les corrélations montrent que ces pays sont susceptibles d'avoir un fort PIB, et de grandes disponibilités en protéines et calories.

In [ ]:
# Projection sur les variables quantitatives
df_acp = df.drop(['Pays'], axis=1)
df_acp.sample()

In [ ]:
# Triangle de corrélations
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(
    df_acp.corr(), 
    mask=np.triu(np.ones_like(df_acp.corr(), dtype=np.bool)), 
    vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Triangle correlation heatmap', fontdict={'fontsize':16}, pad=16)
plt.show()

colinearites = []

## Standardisation des variables

Centrer = normaliser

In [ ]:
# Variables à normaliser et standardiser
variables_non_std = df_acp.drop(['Ratio protéines animales', 'Croissance population', 'Croissance PIB'], axis=1)
variables_non_std.sample()

In [ ]:
# Avec StandardScaler

# Centrage et réduction 
X = variables_non_std.values
std_scale = StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

# Réunion de toutes les variables standardisées
df_acp_standardscaler = pd.merge(
  df_acp.drop(variables_non_std.columns, axis=1), # Variables n'ayant pas nécessité une standardisation
  pd.DataFrame(X_scaled, columns=variables_non_std.columns), # Variables précédemment standardisées
  left_index=True, right_index=True)
df_acp_standardscaler

In [ ]:
# Avec normalize()

# Centrage et réduction 
X = variables_non_std.values
X_scaled = normalize(X)

# Réunion de toutes les variables standardisées
df_acp = pd.merge(
  df_acp.drop(variables_non_std.columns, axis=1), # Variables n'ayant pas nécessité une standardisation
  pd.DataFrame(X_scaled, columns=variables_non_std.columns), # Variables précédemment standardisées
  left_index=True, right_index=True)
df_acp

## Sur 2 composantes

### Sur les variables normalisées avec StandardScaler()

In [ ]:
model = pca(n_components=2)
results = model.fit_transform(df_acp_standardscaler)
fig, ax = model.plot()
plt.show()

In [ ]:
fig, ax = model.biplot(n_feat=3, legend=None)

### Sur les variables normalisées avec normalize()

In [ ]:
model = pca(n_components=2)
results = model.fit_transform(df_acp)
fig, ax = model.plot()
plt.show()

In [ ]:
fig, ax = model.biplot(n_feat=3, legend=None)

In [ ]:
# Dendrogramme sur les 2 composantes de l'ACP
plt.figure(figsize=(15, 5))  
shc.dendrogram(shc.linkage(results['PC'], method='ward'))
plt.axhline(y=1.15, color='black', linestyle='dashed') # Threshold
plt.title('5 clusters envisageables')  
plt.show()

In [ ]:
# Cluster pour chaque individu
cluster = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')  
clusters = pd.DataFrame(cluster.fit_predict(results['PC']), columns=['Cluster'])
deux_composantes = pd.merge(results['PC'], clusters, left_index=True, right_index=True)
deux_composantes = pd.merge(deux_composantes, df['Pays'], left_index=True, right_index=True)
deux_composantes

In [ ]:
# Moyenne par cluster de chaque composante issue de l'ACP (centroïdes)
centroides = deux_composantes.groupby(['Cluster']).mean().reset_index()
centroides['Cluster'] = 'Centroïde'
centroides['Pays'] = 'Centroïde'
centroides

In [ ]:
# Jointure des centroïdes sur la dataframe des 2 composantes
deux_composantes_centroides = pd.concat([deux_composantes, centroides])

# Scatterplot des 2 composantes
plt.figure(figsize=(20,10))
ax = sns.scatterplot(data=deux_composantes_centroides, x='PC1', y='PC2', hue='Cluster', palette=palette, s=250)

def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.02, point['y'], str(point['val']))

label_point(deux_composantes_centroides['PC1'], deux_composantes_centroides['PC2'], deux_composantes_centroides['Pays'], plt.gca())
plt.title('Représentation des 2 composantes, par cluster')
plt.show()

## Sur 3 composantes

In [ ]:
model = pca(n_components=3)
results = model.fit_transform(df_acp)
fig, ax = model.plot()
plt.show()

In [ ]:
fig, ax = model.biplot3d(n_feat=4, legend=None)
plt.show()

In [ ]:
# Dendrogramme sur les 3 composantes de l'ACP
plt.figure(figsize=(15, 5))  
shc.dendrogram(shc.linkage(results['PC'], method='ward'))
plt.axhline(y=1.5, color='black', linestyle='dashed') # Threshold
plt.title('5 clusters envisageables')  
plt.show()

In [ ]:
# Cluster pour chaque individu
cluster = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')  
clusters = pd.DataFrame(cluster.fit_predict(results['PC']), columns=['Cluster'])
trois_composantes = pd.merge(results['PC'], clusters, left_index=True, right_index=True)
trois_composantes

In [ ]:
# Moyenne par cluster de chaque composante issue de l'ACP
centroides = trois_composantes.groupby(['Cluster']).mean().reset_index()
centroides['Cluster'] = 'Centroïde'
centroides

In [ ]:
# Jointure des centroïdes sur la dataframe des 2 composantes
trois_composantes_centroides = pd.concat([trois_composantes, centroides])

# Visualisation des centroïdes
fig = px.scatter_3d(
    pd.concat([trois_composantes_centroides, centroides]), 
    x='PC1', y='PC2', z='PC3', color='Cluster', symbol='Cluster', opacity=.75)
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [ ]:
# Jointure des centroïdes sur la dataframe des 2 composantes
centroides['Cluster'] = '99' # matplot n'autorise que des valeurs numériques dans 'c='
trois_composantes_centroides = pd.concat([trois_composantes, centroides])

# Représentation avec mplot3d
fig = plt.figure(figsize=(10,8))
ax = plt.axes(projection='3d')
ax.scatter3D(
    trois_composantes_centroides['PC1'].values, 
    trois_composantes_centroides['PC2'].values, 
    trois_composantes_centroides['PC3'].values, 
    c=trois_composantes_centroides['Cluster'], s=250, 
    cmap=ListedColormap(palette)
    )
plt.title('Représentation des 3 composantes, par cluster')
plt.show()

# Analyse des clusters

## Résumé

In [ ]:
# Jointure des données non normalisées avec les clusters
df = pd.merge(df, clusters, left_index=True, right_index=True)
df.sample(3)

In [ ]:
# Nombre et exemples de pays dans chaque cluster
for i in df.sort_values('Cluster')['Cluster'].unique():
    print(
        'Cluster', i, ':', 
        len(df[df['Cluster']==i]), 'pays (' + 
        ", ".join(df[df['Cluster']==i].sample(5)['Pays'].values), '...)')

In [ ]:
# Moyenne des variables non standardisées, par cluster
df.groupby(['Cluster']).mean()

## Box plots par variable

**Cluster 0**
- Le plus faible en disponibilité protéique, en ratio de protéines animales et en PIB
- De loin de le plus fort ratio d'import / export de poulet, expliquée par le faible ratio de protéines animales de ces pays

**Cluster 1**
- Les deuxièmes plus forts ratios de protéines animales, disponibilités protéique et calorique, PIB et croissance du PIB
- Les plus faibles populations, croissances de la population et ratios d'import / export de poulet

**Cluster 2**
- Ne dispose que de la plus forte croissance de population
- Toutes les autres variables sont dans la moyenne des autres clusters

**Cluster 3**
- Les plus forts ratios de protéine animale, PIB, disponibilités protéique et calorique 
- Le deuxième plus grand nombre d'habitants
- La plus faible croissance du PIB (s'explique par le haut PIB)

**Cluster 4**
- De loin les plus grands nombres d'habitants
- Le deuxième plus faible PIB
- De loin la plus forte croissance du PIB
- Le deuxième plus fort ratio d'import / export de poulet, loin devant les trois suivants

On cherche des pays à fort ratio de protéines d'origine animales, lui-même positivement corrélé au PIB et aux dispobilités en protéines et calories. Les clusters 3 et 1 apparaissent comme les plus intéressants. 

In [ ]:
sns.boxplot(data=df, y='Dispo_calories', x='Cluster', showfliers=False, palette=palette)
plt.title('Disponibilité calorique par cluster')
plt.show()

In [ ]:
sns.boxplot(data=df, y='Dispo_protéines', x='Cluster', showfliers=False, palette=palette)
plt.title('Disponibilité protéique par cluster')
plt.show()

In [ ]:
sns.boxplot(
    data=df, y='Ratio protéines animales', x='Cluster', palette=palette, showfliers=False,
    showmeans=True, meanprops={'marker':'o', 'markerfacecolor':'white', 'markeredgecolor':'black', 'markersize':'8'})
plt.title('Ratio de protéines animales par cluster')
plt.show()

In [ ]:
sns.boxplot(data=df, y='Millions habitants', x='Cluster', showfliers=False)
plt.title('Nombre d habitants par cluster')
plt.show()

sns.boxplot(data=df.drop(df[df['Cluster']==4].index), y='Millions habitants', x='Cluster', showfliers=False)
plt.title('Nombre d habitants par cluster')
plt.show()

In [ ]:
sns.boxplot(
    data=df, y='Croissance population', x='Cluster', palette=palette, showfliers=False,
    showmeans=True, meanprops={'marker':'o', 'markerfacecolor':'white', 'markeredgecolor':'black', 'markersize':'10'})
plt.title('Croissance de la population par cluster')
plt.show()

In [ ]:
sns.boxplot(data=df, y='PIB', x='Cluster', showfliers=False, palette=palette)
plt.title('PIB par cluster')
plt.show()

In [ ]:
sns.boxplot(data=df, y='Croissance PIB', x='Cluster', showfliers=False, palette=palette)
plt.title('Croissance du PIB par cluster')
plt.show()

In [ ]:
sns.boxplot(data=df, y='Poulet imports / exports', x='Cluster', showfliers=False, palette=palette)
plt.title('Ratio import export de poulet')
plt.show()

sns.boxplot(data=df.drop(df[df['Cluster']==0].index), y='Poulet imports / exports', x='Cluster', showfliers=False, palette=palette)
plt.title('Ratio import export de poulet')
plt.show()

# Analyse du cluster 3

In [ ]:
import geopandas as gpd
import geoplot

In [ ]:
# Dataset naturalearth_lowres
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.rename(columns={'name': 'Pays'}, inplace=True)
world.sort_values(by='Pays').head(10)

In [ ]:
# Dataframe ne contenant que le cluster choisi
pays = df[df['Cluster'].isin([3])]
pays.sort_values(by='Pays')

In [ ]:
# Harmonisation des noms de pays entre les 2 dataframes
world = world.replace('Russia', 'Russian Federation')
world = world.replace('United Kingdom', 'United Kingdom of Great Britain and Northern Ireland')
world = world.replace('Dominican Rep.', 'Dominican Republic')
world = world.replace('Bolivia', 'Bolivia (Plurinational State of)')
world = world.replace('Bosnia and Herz.', 'Bosnia and Herzegovina')
world = world.replace('South Korea', 'Republic of Korea')
world = world.replace('Macedonia', 'North Macedonia')

In [ ]:
# Pays non représentables sur la carte
pays[~pays['Pays'].isin(world['Pays'])]

In [ ]:
# Jointure entre pays et world
pays = world.merge(pays, on='Pays')
pays

In [ ]:
# Nombre de pays par continent
sns.countplot(data=pays, x='continent', palette=palette)
plt.title('Nombre de pays choisis par contintent')
plt.show()

In [ ]:
geoplot.choropleth(
    pays, hue='Dispo_protéines',
    edgecolor='white', linewidth=1,
    cmap='viridis', legend=True,
    scheme='FisherJenks', 
    figsize=(12, 15)
)
plt.title('Disponibilités en protéines dans le cluster 3')
plt.show()

In [ ]:
geoplot.choropleth(
    pays, hue='Ratio protéines animales',
    edgecolor='white', linewidth=1,
    cmap='viridis', legend=True,
    scheme='FisherJenks', 
    figsize=(12, 15)
)
plt.title('Ratio de protéines animales dans le cluster 3')
plt.show()

In [ ]:
geoplot.choropleth(
    pays, hue='PIB',
    edgecolor='white', linewidth=1,
    cmap='viridis', legend=True,
    scheme='FisherJenks', 
    figsize=(12, 15)
)
plt.title('PIB dans le cluster 3')
plt.show()

In [ ]:
geoplot.choropleth(
    pays[pays['continent']=='North America'], hue='Ratio protéines animales',
    edgecolor='white', linewidth=1,
    cmap='viridis', legend=True,
    scheme='FisherJenks', 
    figsize=(12, 15)
)
plt.show()

# Tests

Dans votre partition, vous avez obtenu des groupes distincts. Vérifiez donc qu'ils diffèrent réellement. Pour cela, réalisez les tests statistiques suivants :

- un test d'adéquation : parmi les 4 variables, ou parmi d'autres variables que vous trouverez pertinentes, trouvez une variable dont la loi est normale ;
- un test de comparaison de deux populations (dans le cas gaussien) : choisissez 2 clusters parmi ceux que vous aurez déterminé. Sur ces 2 clusters, testez la variable gaussienne grâce à un test de comparaison.

## Régressions linéaires
Les corrélations sont positives dans chaque cluster. La corrélation avec le ratio de protéines animales est la plus positive.

In [ ]:
# Entre dispo. en protéines et ratio de protéines animales
sns.lmplot(
    data=df, x='Ratio protéines animales', y='Dispo_protéines', hue='Cluster', 
    palette=palette,
    scatter_kws={"s": 100}, height=4, aspect=4/3)
plt.title('Rapport entre disponibilité protéique et ratio de protéines animales, par cluster')
plt.show()

In [ ]:
# Entre dispo. en protéines et dispo. en calories
sns.lmplot(
    data=df, x='Dispo_calories', y='Dispo_protéines', hue='Cluster', 
    palette=palette,
    scatter_kws={"s": 100}, height=4, aspect=4/3)
plt.title('Rapport entre disponibilités protéique et calorique, par cluster')
plt.show()

In [ ]:
# Entre dispo. en protéines et PIB
sns.lmplot(
    data=df, x='PIB', y='Dispo_protéines', hue='Cluster', 
    palette=palette,
    scatter_kws={"s": 100}, height=4, aspect=4/3)
plt.title('Rapport entre disponibilité protéique et PIB, par cluster')
plt.show()

## Normalité

In [ ]:
variable = 'Dispo_protéines' # Variable étudiée

# Test de Shapiro
def test_shapiro(variable):
    stat, p = stats.shapiro(boxcox1p(variable, 1))
    print('Test de Shapiro (stat = %.3f, p-value = %.35f)' % (stat, p))
    if p>.05:
        print('H0 acceptée : distribution probablement normale')
    else:
        print('H0 rejetée : distribution probablement pas normale')
    qqplot(variable, line='s')
    plt.show()

# Test de Shapiro pour chaque cluster
for i in df['Cluster'].unique():
  print('Cluster', i)
  test_shapiro(df[df['Cluster']==i][variable])

## ANOVA

In [ ]:
# Test d'ANOVA entre catégorielle et quantitative
model = smf.ols('Dispo_protéines ~ Cluster', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
p = anova_table['PR(>F)'][0]
print('Test d\'ANOVA\n',
      'comparaison des moyennes de k populations, à partir d\'échantillons aléatoires',
      'et indépendants prélevés dans chacune d\'elles')
#print('Moyennes des modalités :')
#print(sample.groupby('Cluster').mean()['Dispo_calories'])
print('\np-value :', p, '\nstat :', anova_table['F'][0])
if p > 0.05:
    print('H0: the means of the samples are equal.')
else:
    print('H1: one or more of the means of the samples are unequal.')
    
print(    '\nConditions :'
    '\n- les populations étudiées suivent une distribution normale',
    '\n- les variances des populations sont toutes égales (homoscédasticité)')

## Homoscédasticité

In [ ]:
# Nombre de samples, basé sur le nombre d'invididus dans le plus petit cluster
samples = min([len(df[df['Cluster']==3]), len(df[df['Cluster']==1])])

# Valeurs de chaque cluster
a = df[df['Cluster']==3][variable].sample(samples).values
b = df[df['Cluster']==1][variable].sample(samples).values

# Test de Levene
stat, p = stats.levene(a, b)
print('Test de Levene : homoscédasticité (des résidus ?)\n',
    '\nstats :', stat,
    '\np-value :', p)
if p > 0.05:
    print('H0: les variances sont égales')
else:
    print('H1: les variances ne sont pas égales (essayer Welch ANOVA)')
print('\nConditions :'
      '\n- The samples from the populations under consideration are independent',
      '\n- The populations under consideration are approximately normally distributed')   

## Welch

In [ ]:
# Test de Welch entre catégorielle et quantitative
df_welch = df[df['Cluster'].isin([1,3])]
stat, p = stats.ttest_ind(df_welch[variable], df_welch['Cluster'])
print('Test de Welch\n')
print('p-value :', p, '\nstat :', stat)
if p > 0.05:
    print('H0: the means of the samples are equal.')
else:
    print('H1: one or more of the means of the samples are unequal.')

# Références

**Modules**
- [PCA](https://pypi.org/project/pca/)

**Tests**
- [ANOVA à un facteur - Introduction](http://unt-ori2.crihan.fr/unspf/2010_Limoges_Vignoles_StatsAnova/co/09-1-1-introduction.html)


**ACP**
- [Cours d'ACP : théorie et pratique](https://www.youtube.com/watch?v=8qw0bNfK4H0) (Youtube)
- [A Beginner’s Guide to Hierarchical Clustering and how to Perform it in Python](https://www.analyticsvidhya.com/blog/2019/05/beginners-guide-hierarchical-clustering/)